In [ ]:
%%file EEGStaticMethods.m
classdef EEGStaticMethods
    % A class which is used for generating parameters.
    %
    % EEGStaticMethods Properties:
    %
    % EEGStaticMethods Methods:
    %   rawMom - method calculating moments;
    %   rawNrm - method calculating norms;
    %   rawAdjTotSNRdB - method for adjusting signal to noise ratio;
    %   rawCartToSph - method converting cartesian to spherical coordinates;
    %   rawSphToCart - method converting spherical to cartesian coordinates;
    %   rawIsStableMVAR - method checking is MVAR model is stable;
    %   diagonMask - method produces masking square array;
    %   stableMVAR - method creating stble MVAR matrix;
    %   rawPlotPDC - method plotting PDC;
    %   rawImgSC - pretty image;
    %   rawHotColdColorMap - pretty color-map for MATLAB figures;
    %   rawDispA00 - method plotting A00 matrix;
    %   rawPlotA00 - method plotting A00 matrix via PDC;
    %   ccdisp - prints cybercraft;
    %   ccrender - method for rendering plots.

    properties(GetAccess='public', SetAccess='public')
    end
    methods
        function obj = EEGStaticMethods()
            addpath('~/toolboxes/arfit');
        end
    end
    methods
        function [y] = rawNrm(obj, x)
             % Compute norm of the random vector.

             y = sqrt(sum(obj.rawMom(x, 2), 2));
        end
        function [y] = rawAdjTotSNRdB(obj, x01, x02, newSNR)
            % Adjusts signal power according to predefined SNR ratio.

            y = ((x02 / obj.rawNrm(x02)) * obj.rawNrm(x01)) / (db2pow(0.5 * newSNR));
        end
        function rawDispA00(obj, A00)
    
            obj.rawImgSC(A00, 8);
            tmp_base = 255;
            tmp_colorMapMat = obj.rawHotColdColorMap(tmp_base);
            caxis([-max(abs(min(min(A00))), abs(max(max(A00)))) max(abs(min(min(A00))), abs(max(max(A00))))]);
            colormap(tmp_colorMapMat);
            colorbar;
            hold on;
            tmp_stem = 0.5 + size(A00, 1):size(A00, 1):size(A00, 2) - 0.5;
            tmp_vals = size(A00, 1) + 0.5*ones(size(tmp_stem));
            stem(tmp_stem, tmp_vals, 'Color', 'k', 'LineWidth', 0.5, 'Marker', 'none');
            set(gca, 'XTick', [0: size(A00, 1): size(A00, 2)]);
        end
        function rawPlotA00PDC(obj, A00, name)

            SETUP.PDC_RES = [0:0.01:0.5];
            [mDTF, mPDC] = obj.myPDC(A00, SETUP.PDC_RES);
            obj.rawPlotC(abs(mPDC), SETUP.PDC_RES, strcat(name, " ", "PDC"));
        end
        function rawPlotA00DTF(obj, A00, name)

            SETUP.PDC_RES = [0:0.01:0.5];
            [mDTF, mPDC] = obj.myPDC(A00, SETUP.PDC_RES);
            obj.rawPlotC(abs(mDTF), SETUP.PDC_RES, strcat(name, " ", "DTF"));
        end
        function [] = ccrender(obj, varargin)
            %%
            % CCRENDER --- figure rendering settings, usage:
            %
            % ccrender( axisLimits, finish, origin, originLimits )
            %
            %  - axisLimits [optional]:
            %      limits display to specified range defined using either:
            %        - vector of length 2 (all axes have same limits)
            %          [ xyzLim_1, xyzLim_2 ],
            %        - vector of length 6 (each axis has its own limits)
            %          [ xLim_1, xLim_2, yLim_1, yLim_2, zLim_1, zLim_2 ];
            %  - finish [optional]:
            %      changes rendering scheme;
            %  - origin [optional]:
            %      true/false;
            %  - originLim [optional]:
            %      a number.
            %
            % Questions and comments: nikadon@gmail.com
            %
            % TODO:
            % - add "try" envelopes for major blocks
            %
            %

            p = inputParser;

            defAxisLimits = [];
            %   chkAxisLimits = @(x) validateattributes(x,{'numeric'},{'vector','numel',2});
            chkAxisLimits = @(x) isnumeric(x) && isvector(x) && (length(x) == 2 || length(x) == 6);
            addOptional(p, 'axisLimits', defAxisLimits, chkAxisLimits)

            defOriginLimits = [];
            chkOriginLimits = @(x) isnumeric(x) && isvector(x) && (length(x) == 2 || length(x) == 6);
            addOptional(p, 'originLimits', defOriginLimits, chkOriginLimits)

            defEqualize = false;
            chkEqualize = @(x) validateattributes(x, {'logical'}, {});
            addParameter(p, 'equalize', defEqualize, chkEqualize)

            defOrigin = true;
            chkOrigin = @(x) validateattributes(x, {'logical'}, {});
            addParameter(p, 'origin', defOrigin, chkOrigin)

            defLabels = true;
            chkLabels = @(x) validateattributes(x, {'logical'}, {});
            addParameter(p, 'labels', defLabels, chkLabels)

            defFinish = 'matte';
            vldFinish = {'glossy', 'matte'};
            chkFinish = @(x) any(validatestring(x, vldFinish));
            addParameter(p, 'finish', defFinish, chkFinish)

            s = dbstack;

            p.KeepUnmatched = true;

            parse(p, varargin{:})

            tmp_gcf = gcf

            obj.ccdisp('INIT: ccrender')
            obj.ccdisp(['Updating figure: ', num2str(tmp_gcf.Number), '...'])

            if ~isempty(p.UsingDefaults)
                obj.ccdisp([ s(1).name, ': Using defaults:'])
                disp(p.UsingDefaults')
            end
            if ~isempty(p.Results)
                obj.ccdisp([ s(1).name, ': Results:'])
                disp(p.Results')
            end
            if ~isempty(fieldnames(p.Unmatched))
                obj.ccdisp([ s(1).name, ': Extra (unmatched) inputs:'])
                disp(p.Unmatched')
            end

            %%

            if length(p.Results.axisLimits) == 2
                if p.Results.axisLimits(1) >= p.Results.axisLimits(2)
                    help ccrender;
                    error('CYBERCRAFT: axis limits [xyzLim_1,xyzLim_2] must be increasing.');
                else
                    axis([ p.Results.axisLimits(1) p.Results.axisLimits(2) p.Results.axisLimits(1) p.Results.axisLimits(2) p.Results.axisLimits(1) p.Results.axisLimits(2) ]);
                end
            elseif length(p.Results.axisLimits) == 6
                if p.Results.axisLimits(1) >= p.Results.axisLimits(2)  ||  p.Results.axisLimits(3) >= p.Results.axisLimits(4)  ||  p.Results.axisLimits(5) >= p.Results.axisLimits(6)
                    help ccrender;
                    error('CYBERCRAFT: axis limits [xLim_1,xLim_2,yLim_1,yLim_2,zLim_1,zLim_2] must be increasing for each axis.');
                else
                    axis([ p.Results.axisLimits(1) p.Results.axisLimits(2) p.Results.axisLimits(3) p.Results.axisLimits(4) p.Results.axisLimits(5) p.Results.axisLimits(6) ]);
                end
            end
            %{
            xlim([-15,15])
            ylim([-15,15])
            zlim([-15,15])
            axis square
            axis equal
            axis auto
            %}

            if p.Results.equalize == true
                xMin = min(xlim);
                xMax = max(xlim);
                yMin = min(ylim);
                yMax = max(ylim);
                zMin = min(zlim);
                zMax = max(zlim);
                xyzMin = min([ xMin yMin zMin ]);
                xyzMax = max([ xMax yMax zMax ]);
                axis([ xyzMin xyzMax xyzMin xyzMax xyzMin xyzMax ]);
            end

            if p.Results.labels == true
                xlabel('[mm]');
                ylabel('[mm]');
                zlabel('[mm]');
            end

            set(gca,'DataAspectRatio',   [1 1 1]);
            set(gca,'PlotBoxAspectRatio',[1 1 1]);

            camproj('perspective'); % perspective | ortographic

            %{
            set(gca,'Projection','ortographic') % problem here?
                                                %}

            set(gca,'CameraViewAngle', 10);

            az = 60;
            el = 30;
            view(az, el);

            if strcmp(p.Results.finish, 'glossy')
                lighting gouraud;
                material shiny;
                camlight;
                %{
                shading interp;
                light;
                lighting phong;
                %}
            end

            axis on;
            box on;

            %%
            if p.Results.origin
                if length(p.Results.originLimits) == 2
                    if p.Results.originLimits(1) > 0
                        help ccrender;
                        error('CYBERCRAFT: origin axis low limit must be negative number.')
                    elseif p.Results.originLimits(2) < 0
                        help ccrender;
                        error('CYBERCRAFT: origin axis higher limit must be positive number.')
                    else
                        xMin = p.Results.originLimits(1);
                        xMax = p.Results.originLimits(2);
                        yMin = p.Results.originLimits(1);
                        yMax = p.Results.originLimits(2);
                        zMin = p.Results.originLimits(1);
                        zMax = p.Results.originLimits(2);
                    end
                elseif length(p.Results.originLimits) == 6
                    if p.Results.originLimits(1) > 0  ||  p.Results.originLimits(3) > 0  || p.Results.originLimits(5) > 0
                        help ccrender;
                        error('CYBERCRAFT: origin axis low limit must be negative number.')
                    elseif p.Results.originLimits(2) < 0  ||  p.Results.originLimits(4) < 0  ||  p.Results.originLimits(6) < 0
                        help ccrender;
                        error('CYBERCRAFT: origin axis higher limit must be positive number.')
                    else
                        xMin = p.Results.originLimits(1);
                        xMax = p.Results.originLimits(2);
                        yMin = p.Results.originLimits(3);
                        yMax = p.Results.originLimits(4);
                        zMin = p.Results.originLimits(5);
                        zMax = p.Results.originLimits(6);
                    end
                else
                    xMin = min(xlim);
                    xMax = max(xlim);
                    yMin = min(ylim);
                    yMax = max(ylim);
                    zMin = min(zlim);
                    zMax = max(zlim);
                end

                hold on

                try
                    ccfgFigH = evalin( 'base', 'ccfgFigH' );
                catch
                    ccfgFigH = [];
                end

                try
                    tmp_fields = fieldnames(ccfgFigH(tmp_gcf.Number).origin);
                    for ii = 1:length(tmp_fields)
                        try
                            delete(ccfgFigH(tmp_gcf.Number).origin.(tmp_fields{ii}));
                        catch tmp_catched_2
                            fprintf('\n\nPSEUDO-WARNING[2]: %s\n\n', tmp_catched_2.message);
                        end
                    end
                catch tmp_catched_1
                    fprintf('\n\nPSEUDO-WARNING [1]: %s\n\n', tmp_catched_1.message);
                end

                ccfgFigH(tmp_gcf.Number).origin.xp = quiver3( 0, 0, 0, 1.1 * xMax, 0,    0,    'color', [1 0 0], 'LineWidth', 1.0, 'LineStyle', '-' ); hold on;
                ccfgFigH(tmp_gcf.Number).origin.yp = quiver3( 0, 0, 0, 0,    1.1 * yMax, 0,    'color', [0 1 0], 'LineWidth', 1.0, 'LineStyle', '-' ); hold on;
                ccfgFigH(tmp_gcf.Number).origin.zp = quiver3( 0, 0, 0, 0,    0,    1.1 * zMax, 'color', [0 0 1], 'LineWidth', 1.0, 'LineStyle', '-' ); hold on;

                ccfgFigH(tmp_gcf.Number).origin.xn = quiver3( 0, 0, 0, xMin, 0,    0,    'color', [1 0 0], 'LineWidth', 1.0, 'LineStyle', '-.', 'ShowArrowHead', 'off' ); hold on;
                ccfgFigH(tmp_gcf.Number).origin.yn = quiver3( 0, 0, 0, 0,    yMin, 0,    'color', [0 1 0], 'LineWidth', 1.0, 'LineStyle', '-.', 'ShowArrowHead', 'off' ); hold on;
                ccfgFigH(tmp_gcf.Number).origin.zn = quiver3( 0, 0, 0, 0,    0,    zMin, 'color', [0 0 1], 'LineWidth', 1.0, 'LineStyle', '-.', 'ShowArrowHead', 'off' ); hold on;

                ccfgFigH(tmp_gcf.Number).origin.xt = text(   1.2 * xMax, 0,        0,        'OX', 'color', [1 0 0] );
                ccfgFigH(tmp_gcf.Number).origin.yt = text(   0,        1.2 * yMax, 0,        'OY', 'color', [0 1 0] );
                ccfgFigH(tmp_gcf.Number).origin.zt = text(   0,        0,        1.2 * zMax, 'OZ', 'color', [0 0 1] );

                assignin( 'base', 'ccfgFigH', ccfgFigH );

            end

            %%

            xLimits = xlim;
            yLimits = ylim;
            zLimits = zlim;

            obj.ccdisp( [ 'xLimits: [', num2str(xLimits(1)), ', ', num2str(xLimits(2)), ']' ] )
            obj.ccdisp( [ 'yLimits: [', num2str(yLimits(1)), ', ', num2str(yLimits(2)), ']' ] )
            obj.ccdisp( [ 'zLimits: [', num2str(zLimits(1)), ', ', num2str(zLimits(2)), ']' ] )

            grid on;
            rotate3d on;
            hold on;

            obj.ccdisp(['Figure: ',num2str(tmp_gcf.Number),' has been updated!'])

            obj.ccdisp('DONE: ccrender')
        end
    end
    methods(Static)
        function [M] = rawMom(x01, k)
            % Compute raw moment of the k-th order for the random vector.

            M = mean(x01.^k, 1);
        end
        function [y] = rawCartToSph(x)
            % Convert cartesian to spherical coordinates (row-wise)

            if ~isempty(x)
                for ii = 1:size(x, 1)
                    [y(ii,1), y(ii,2), y(ii,3)] = cart2sph(x(ii,1), x(ii,2), x(ii,3));
                end
            else
                y = x;
            end
        end
        function [y] = rawSphToCart(x)
            % Convert spherical to cartesian coordinates (row-wise)

            if ~isempty(x)
                for ii = 1:size(x,1)
                    [y(ii,1), y(ii,2), y(ii,3)] = sph2cart(x(ii,1), x(ii,2), x(ii,3));
                end 
            else    
                y = x;
            end     
        end
        function [H, varargout] = rawIsStableMVAR(A00, varargin)
            % Check stability of generated MVAR model.

            if isempty(varargin)
                STAB = 1;
                disp(['Checking if eigenvalues are inside unit circle.']);
            elseif length(varargin) == 1
                STAB = varargin{1};
                disp(['Checking if eigenvalues are inside circle with radius of ', num2str(STAB), '.']);
            else
                error('Too many input arguments');
            end
            S00          = size(A00, 1);                  % dimension of state vectors
            P00          = size(A00, 2)/S00;              % order of process
            tmp_lambda   = eig([A00; eye((P00 - 1)*S00) zeros((P00 - 1) * S00, S00)]); % Haufe, "Towards EEG source connectivity analysis", p. 17
            % Lutkepohl, Helmut, Also New Introduction to Multiple Time Series Analysis, 2005, p. 15
            H            = ~any(abs(tmp_lambda) > STAB);
            varargout{1} = max(abs(tmp_lambda));
        end
        function M00 = diagonMask(S00, M00_frc)
            % diagonMask produces masking square array M (SxS) whose all diagonal
            % and some off-diagonal elements are ones. All othere elements are zeros.
            % The off-diagonal elements are sampled pseudo-randomly and their number is declared by
            % second argument of this function which should be a number between zero and one that
            % describes the proportion of ones to zeros among all off-diagonal elements
            % of the resulting array.
            %
            % Usage:
            %
            %    MSK = diagonMask(S,frac)
            %
            %    This file is free software and a part of CyberCraft Toolkit.
            %    You can redistribute it and/or modify it under the
            %    terms of the GNU General Public License as published by the Free Software Foundation,
            %    either version 3 of the License, or (at your option) any later version.
            %
            %    CyberCraft is distributed in the hope that it will be useful,
            %    but WITHOUT ANY WARRANTY; without even the implied warranty of
            %    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
            %    GNU General Public License for more details.
            %
            %    You should have received a copy of the GNU General Public License
            %    along with CyberCraft. If not, see <http://www.gnu.org/licenses/>.
            %
            %    % Example usage:
            %    S00     = 10;
            %    M00_frc = 0.1;
            %    M00     = diagonMask(S00, M00_frc)
            %    figure(1);
            %    clf;
            %    imagesc(M00);
            %    colorbar;
            %    (numel(find(M00)) - S00) / (numel(M00) - S00)
            %    imagesc(sim_sig00.M00);
            %    colorbar;

            M00     = eye(S00);
            M00_idx = find(~M00);
            M00_smp = datasample(M00_idx, round(M00_frc * numel(M00_idx)), 'Replace', false);
            M00(M00_smp) = deal(1);
        end
        function A00 = stableMVAR(S00, P00, M00, set_RNG, set_STAB, set_ITER, DEBUG)
            % Procedure inspired by function stablemvar
            % that is a part of MVARICA Toolbox.
            % It generates composite stable MVAR model matrix A00 representing WSS process.

            tmp_iterNow = 0;
            tmp_lambda = Inf;
            while any(abs(tmp_lambda) > set_STAB) && tmp_iterNow < set_ITER
                tmp_V = orth(rand(S00 * P00, S00 * P00));
                tmp_U = orth(rand(S00 * P00, S00 * P00));
                lambdatmp = set_RNG(1) + (set_RNG(2) - set_RNG(1)) * rand(S00 * P00, 1);
                tmp_A00 = tmp_V * diag(lambdatmp) * tmp_U';
                A00 = tmp_A00(1:S00, :);
                A00 = A00 .* repmat(M00, 1, P00); % nulling of some coefficients based on mask
                tmp_lambda = eig([A00; eye((P00 - 1) * S00) zeros((P00 - 1)*S00, S00)]); % Look in Haufe, "Towards EEG source connectivity analysis", p. 17
                tmp_iterNow = tmp_iterNow + 1;
            end
            if tmp_iterNow >= set_ITER
                A0 = [];
                error('Could not generate stable MVAR model in given number of iterations (set_ITER)');
            end
            if DEBUG
               S00 = sim_sig00.S00;
               P00 = sim_sig00.P00;
               w00 = sim_sig00.w00;
               C00 = sim_sig00.C00;
               n00 = sim_sig00.n00;
               K00 = sim_sig00.K00;
            end
        end
        function rawPlotC(pdcData, accf, name)

            tmp_row  = size(pdcData, 1);
            tmp_col  = size(pdcData, 2);
            tmp_cnt  = 0;
            for i = 1:tmp_row
                for j = 1:tmp_col
                    tmp_cnt = tmp_cnt + 1;
                    subplot(tmp_row, tmp_col, tmp_cnt);
                    area(accf, squeeze(pdcData(i, j, :)));
                    axis tight;
                    ylim([0, 1]);
                end
            end
            currentFigure = gcf;
            title(currentFigure.Children(end), name);
        end
        function [h1, h2] = rawImgSC(mat, varargin)
    
            h1 = imagesc(mat);
            [x, y] = ndgrid(1:size(mat, 2), 1:size(mat, 1));
            mat = mat';
            if ~isempty(varargin)
                fSize = varargin{1};
            else
               fSize = 18;
            end
            h2 = text(x(:), y(:), strtrim(cellstr(num2str(mat(:), '%.2f'))), 'HorizontalAlignment', 'center', 'FontSize', fSize);
        end
        function [y] = rawHotColdColorMap(x)
            % Pretty color-map for MATLAB figures.
            % [y] = rawHotColdColorMap(x)
            % Suggested rationalization: base/2

            y = [linspace(0, 1, x)', linspace(0, 1, x)', linspace(1, 1, x)'];
            y = [y; fliplr(flipud(y))];
            % imagesc(permute(y,[1,3,2]))
        end
        function [ ] = ccdisp(str)
            % CCDISP display as CYBERCRAFT
            %
            % Use
            %
            %   ccdisp( str )
            %

            % add here "try + catch" OR "ifexists"

            disp(['CYBERCRAFT: ', str]);

        end
        function [DTF, PDC] = myPDC(A00, f, N)

            M = size(A00, 1); % A00 has dim M*pM
            p = size(A00, 2)/M; % p is the order of the MVAR model

            if nargin < 2
                N = 32;
            end;
            N = length(f);
            s = exp(i*2*pi*f); % vector of complex exponentials
            z = i*2*pi;

            % Spectral matrices have M rows, M columns. They are calculated for N frequencies.
            H = zeros(M, M, N); % Transfer Matrix
            DTF = zeros(M, M, N); % Directed transfer function
            PDC = zeros(M, M, N); % Partial directed coherence

            % Auxiliary rows vectors
            denomDTF = zeros(M, 1); % Column denominator for DC
            denomPDC = zeros(M, 1)'; % Row denominator for PDC

            A = [eye(M) -A00]; % matrix from which M*M blocks are selected to calculate spectral functions

            % Computation of spectral functions
            for n = 1 : N, % at each frequency \lambda = f(n)

                % Coefficient matrix in the frequency domain
                As = zeros(M, M); % = A(\lambda)
                for k = 1 : p + 1,
                    % Slicing matrix A
                    As = As + A(:, k*M + (1 - M:0))*exp(-z*(k - 1)*f(n));
                end;
                % Since A consists of identity matrix and MINUS A00 matrix As we become
                % I - \sum_{k = 1}^{p} A_s \exp(2 i k \pi \lambda)

                % Transfer matrix
                H(:, :, n)  = inv(As);

                % Denominators of PDC, DTF, GPDC for each m = 1, ..., M channels
                for m = 1 : M,
                    denomDTF(m) = sqrt((abs(H(m, :, n)).^2) * ones(M, 1));
                    mthcolumnAs = squeeze(As(:, m)); % m-th column of As...
                    denomPDC(m) = sqrt(mthcolumnAs' * mthcolumnAs);
                end;

                % Directed Transfer Function
                DTF(:, :, n) = H(:, :, n) ./ denomDTF(:, ones(M, 1));

                % Partial Directed Coherence
                PDC(:, :, n) = As ./ denomPDC(ones(1, M), :);
            end;
        end
    end
end